In [ ]:
### Import libraries
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
## Read data and rename columns ###
orig_data = pd.read_excel("OppScrData.xlsx")
for col in orig_data.columns:
    new_col = '_'.join(col.split())
    orig_data.rename(columns={col:new_col}, inplace=True)

# Predicting Death (DEATH_[d_from_CT])

In [ ]:
## Data setup for predicting 'DEATH_[d_from_CT]'

data = orig_data.copy()

data['BMI_>30'] = np.where(data['BMI_>30']=='Y', 1, 0)
data.rename(columns={'BMI_>30':'BMI_more_than_30'}, inplace=True)
data['Sex'] = np.where(data['Sex']=='Male', 1, 0)
data['Tobacco'] = np.where(data['Tobacco']=='Yes', 1, 0)

## Zero the NULL values
data['Alcohol_abuse'] = np.where(data['Alcohol_abuse'].isna(),0,1)


### Convert percentage values into uniform numeric values
col = 'FRS_10-year_risk_(%)'
data[col] = data[col].astype(str)
data[col] = data[col].str.replace('X','')
data[col] = data[col].str.replace('<1%',str(np.random.uniform(0, 0.01)))
data[col] = data[col].str.replace('>30%',str(np.random.uniform(0.3, 0.99)))
data[col] = pd.to_numeric(data[col], errors='coerce')
data[col] = data[col].astype(float)
med = data[col].median()
data[col] = data[col].fillna(med)
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)


## clip outliers to 95th percentile
for col in ['FRAX_10y_Fx_Prob_(Orange-w/_DXA)','FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)']:
    data[col] = data[col].astype(str)
    data[col] = data[col].str.replace('_','')
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col] = data[col].astype(float)
    med = data[col].median()
    data[col] = data[col].fillna(med)
    upper_limit = data[col].quantile(0.95)
    data[col] = data[col].clip(upper_limit)

    
data['Met_Sx'] = np.where(data['Met_Sx']=='Y', 1, np.where(data['Met_Sx']=='N', 0, -1))


data['CVD'] = np.where(data['CVD_DX'].isna(),0,1)
data['MI'] = np.where(data['MI_DX'].isna(),0,1)
data['Heart_failure'] = np.where(data['Heart_failure_DX'].isna(),0,1)


## The main column we are predicting is stored in 'days'
data['days'] = data['DEATH_[d_from_CT]']

data['diabetes'] = np.where(data['Type_2_Diabetes_DX'].isna(), 0, 1)


data['L1_HU_BMD'] = data['L1_HU_BMD'].clip(lower=0)

## Created a new column of Fat are/ Total body area
data['TAT/Body_Area'] = data['TAT_Area_(cm2)']/data['Total_Body_Area_EA_(cm2)']
col = 'VAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'SAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'VAT/SAT_Ratio'
upper_limit = data[col].quantile(0.95)
data[col] = data[col].clip(upper_limit)

data['AoCa_Agatston'] = data['AoCa_Agatston'].clip(upper=data['AoCa_Agatston'].quantile(0.99))
data['AoCa_Agatston'] = data['AoCa_Agatston'].fillna(data['AoCa_Agatston'].median())
data['AoCa_Agatston'] = np.log(data['AoCa_Agatston']+1)

data['Liver_HU_(Median)'] = pd.to_numeric(data['Liver_HU_(Median)'], errors='coerce')
data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].astype(float)
#data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].fillna(data['Liver_HU_(Median)'].median())
                          
for col in ['BMI','TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'VAT_Area_(cm2)','SAT_Area_(cm2)',
            'Muscle_Area_(cm2)','TAT/Body_Area']:
    data[col] = data[col].fillna(data[col].median())

### Below features filled by iterative imputing

impute_it = IterativeImputer()

col = 'L1_HU_BMD'
rel_cols = ['Age_at_CT', 'Muscle_HU', 'L1_HU_BMD']
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:, 2]

### VAT/SAT ratio -- Muscle area, Sex

col = 'VAT/SAT_Ratio'
rel_cols = ['Muscle_Area_(cm2)', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]


col = 'Muscle_HU'
rel_cols = ['Age_at_CT', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]

col = 'L3_SMI_(cm2/m2)'
rel_cols = ['BMI_more_than_30', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]
                       

col = 'Liver_HU_(Median)'                                     
rel_cols = ['BMI_more_than_30', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]


In [ ]:
cols = ['Record_ID', 'Clinical_F/U_interval_[d_from_CT]', 'BMI', 'BMI_more_than_30', 'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse',
       'FRS_10-year_risk_(%)', 'FRAX_10y_Fx_Prob_(Orange-w/_DXA)',
       'FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx',
       'days', 'CVD', 'Heart_failure', 'MI', 'diabetes', 'L1_HU_BMD',
       'TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

data = data[cols].copy()
data['death_binary'] = np.where(data['days'].isna(),0,1)

In [ ]:
train_idx = pd.read_csv('Patients(RECORD_ID)TrainData.csv')['Record_ID']
test_idx = pd.read_csv('Patients(RECORD_ID)_TestData.csv')['Record_ID']
train = data[data['Record_ID'].isin(train_idx)]
test = data[data['Record_ID'].isin(test_idx)]

In [ ]:
# Train on Non-Null values
train = train[~train['days'].isna()]
val = test[~test['days'].isna()]

In [ ]:
## Create separate sets of train and test based on category of features picked.

all_feats = ['BMI', 'BMI_more_than_30',
       'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse', 'FRS_10-year_risk_(%)',
       'FRAX_10y_Fx_Prob_(Orange-w/_DXA)','FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx', 'CVD',
       'Heart_failure', 'MI', 'diabetes', 'L1_HU_BMD', 'TAT_Area_(cm2)',
       'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']


ct_feats = ['L1_HU_BMD', 'TAT_Area_(cm2)',
       'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

feats = all_feats ## To pick all the features
#feats = [x for x in all_feats if x not in ct_feats] ## To pick only clinical features
#feats = ct_feats  ## To pick only CT features

label = 'days'
X_train = train[feats].values
Y_train = train[label].values
X_val = val[feats].values
Y_val = val[label].values
X_test = test[feats].values
Y_test = test[label].values
print(X_test.shape)
print(X_val.shape)
print(Y_val.shape)

## Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)

## XGBoost

In [ ]:
eval_set = [(X_train, Y_train), (X_val, Y_val)]
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.01, subsample=1, colsample_bytree=1)
model.fit(X_train, Y_train, eval_metric='rmse', eval_set=eval_set)

In [ ]:
results = model.evals_result()
plt.plot(results['validation_0']['rmse'], label='train')
plt.plot(results['validation_1']['rmse'], label='test')
plt.legend()
plt.xlabel('Estimators')
plt.ylabel('RMSE')
plt.show()

In [ ]:
variables_df = pd.DataFrame(feats, columns=['Variables'])
importance_score_df = pd.DataFrame(model.feature_importances_, columns=['Importance Score'])

# print(variables_df)
# print(importance_score_df)

feature_imp = pd.concat([variables_df, importance_score_df], axis=1)
feature_imp = feature_imp.sort_values(by='Importance Score', ascending=False)
feature_imp = feature_imp.reset_index()
x = 'Importance Score'
y = 'Variables'
sns.barplot(x=x, y=y, data=feature_imp[:10], orient="h", palette="Blues_r")
plt.title('Feature Importance')
plt.show()

## Support Vector Regressor

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
model.fit(X_train, Y_train)

In [ ]:
## Predict on Non NULL test samples

preds = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(Y_val, preds))
print("RMSE=",rmse, "Test samples mean=", Y_val.mean())

print(Y_val.shape)
tmp = {'Y_test':Y_val,'preds':preds} 
tmp_df = pd.DataFrame(tmp)
tmp_df.sort_values(by='Y_test', inplace=True)
sns.scatterplot(data=tmp_df, x='Y_test', y='preds')

In [ ]:
# Prediting on Null values

test_null = test[test['days'].isna()]
X_test_null = test_null[feats].values
preds_test_null = model.predict(X_test_null)

tmp_df = pd.DataFrame(preds)
# print(tmp_df.describe())
tmp_df = pd.DataFrame(preds_test_null)
# print(tmp_df.describe())

all_preds = model.predict(X_test)
tmp_df = pd.DataFrame(all_preds)
# print(tmp_df.describe())


tmp = {'Y_test':Y_test,'preds':all_preds}
tmp_df = pd.DataFrame(tmp)
y_nz = tmp_df[~tmp_df['Y_test'].isna()]['Y_test']
preds_nz = tmp_df[~tmp_df['Y_test'].isna()]['preds']
rmse = math.sqrt(mean_squared_error(y_nz, preds_nz))
print("Error% for Non-zero values: {:0.4f}".format((rmse/y_nz.mean())))

In [ ]:
## categorize non null and null labels separately.

tmp = {'Y_test':Y_test,'preds':all_preds}
tmp_df = pd.DataFrame(tmp)
tmp_df['Y_test_cat'] = np.where(tmp_df['Y_test'].isna(), 0, 1)
sns.boxplot(data=tmp_df, x='Y_test_cat', y='preds')

# Predicting Diabetes days from CT

In [ ]:
## data setup for predicting diabetes

data = orig_data.copy()

data['BMI_>30'] = np.where(data['BMI_>30']=='Y', 1, 0)
data.rename(columns={'BMI_>30':'BMI_more_than_30'}, inplace=True)
data['Sex'] = np.where(data['Sex']=='Male', 1, 0)
data['Tobacco'] = np.where(data['Tobacco']=='Yes', 1, 0)

data['Alcohol_abuse'] = np.where(data['Alcohol_abuse'].isna(),0,1)

col = 'FRS_10-year_risk_(%)'
data[col] = data[col].astype(str)
data[col] = data[col].str.replace('X','')
data[col] = data[col].str.replace('<1%',str(np.random.uniform(0, 0.01)))
data[col] = data[col].str.replace('>30%',str(np.random.uniform(0.3, 0.99)))
data[col] = pd.to_numeric(data[col], errors='coerce')
data[col] = data[col].astype(float)
med = data[col].median()
data[col] = data[col].fillna(med)
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)

for col in ['FRAX_10y_Fx_Prob_(Orange-w/_DXA)','FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)']:
    data[col] = data[col].astype(str)
    data[col] = data[col].str.replace('_','')
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col] = data[col].astype(float)
    med = data[col].median()
    data[col] = data[col].fillna(med)
    upper_limit = data[col].quantile(0.95)
    data[col] = data[col].clip(upper_limit)

data['Met_Sx'] = np.where(data['Met_Sx']=='Y', 1, np.where(data['Met_Sx']=='N', 0, -1))


data['CVD'] = np.where(data['CVD_DX'].isna(),0,1)
data['MI'] = np.where(data['MI_DX'].isna(),0,1)
data['Heart_failure'] = np.where(data['Heart_failure_DX'].isna(),0,1)

data['days'] = data['Type_2_Diabetes_DX_Date_[d_from_CT]']
data['diabetes'] = np.where(data['Type_2_Diabetes_DX'].isna(), 0, 1)

data['L1_HU_BMD'] = data['L1_HU_BMD'].clip(lower=0)
data['TAT/Body_Area'] = data['TAT_Area_(cm2)']/data['Total_Body_Area_EA_(cm2)']
col = 'VAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'SAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'VAT/SAT_Ratio'
upper_limit = data[col].quantile(0.95)
data[col] = data[col].clip(upper_limit)

data['AoCa_Agatston'] = data['AoCa_Agatston'].clip(upper=data['AoCa_Agatston'].quantile(0.99))
data['AoCa_Agatston'] = data['AoCa_Agatston'].fillna(data['AoCa_Agatston'].median())
data['AoCa_Agatston'] = np.log(data['AoCa_Agatston']+1)

data['Liver_HU_(Median)'] = pd.to_numeric(data['Liver_HU_(Median)'], errors='coerce')
data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].astype(float)
data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].fillna(data['Liver_HU_(Median)'].median())
                          
for col in ['BMI','TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'VAT_Area_(cm2)','SAT_Area_(cm2)',
            'Muscle_Area_(cm2)','TAT/Body_Area']:
    data[col] = data[col].fillna(data[col].median())


### Below features filled by iterative imputing

impute_it = IterativeImputer()

col = 'L1_HU_BMD'
# ind = data.index[data[col].isna()].tolist()
#print(len(data.loc[data[col] < 100, col]))
#print(data.loc[data[col].isna(), 'L1_HU_BMD'])
rel_cols = ['Age_at_CT', 'Muscle_HU', 'L1_HU_BMD']
X = data[rel_cols]
X = impute_it.fit_transform(X)
# print(X[:, 2])
data[col] = X[:, 2]
# print(col, data[col].isna().sum())

### VAT/SAT ratio -- Muscle area, Sex

col = 'VAT/SAT_Ratio'
rel_cols = ['Muscle_Area_(cm2)', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]


col = 'Muscle_HU'
rel_cols = ['Age_at_CT', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]

col = 'L3_SMI_(cm2/m2)'
rel_cols = ['BMI_more_than_30', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]
                       

col = 'Liver_HU_(Median)'                                     
rel_cols = ['BMI_more_than_30', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]

In [ ]:
cols = ['Record_ID', 'BMI', 'BMI_more_than_30', 'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse',
       'FRS_10-year_risk_(%)', 'FRAX_10y_Fx_Prob_(Orange-w/_DXA)',
       'FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx',
       'days', 'CVD', 'Heart_failure', 'MI', 'L1_HU_BMD',
       'TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

data = data[cols].copy()

In [ ]:
train_idx = pd.read_csv('Patients(RECORD_ID)TrainData.csv')['Record_ID']
test_idx = pd.read_csv('Patients(RECORD_ID)_TestData.csv')['Record_ID']
train = data[data['Record_ID'].isin(train_idx)]
test = data[data['Record_ID'].isin(test_idx)]

In [ ]:
# Train on Non-Null values
train = train[~train['days'].isna()]
val = test[~test['days'].isna()]

In [ ]:
feats = ['BMI', 'BMI_more_than_30', 'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse',
       'FRS_10-year_risk_(%)', 'FRAX_10y_Fx_Prob_(Orange-w/_DXA)',
       'FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx',
        'CVD', 'Heart_failure', 'MI', 'L1_HU_BMD',
       'TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

label = 'days'
X_train = train[feats].values
Y_train = train[label].values
X_val = val[feats].values
Y_val = val[label].values

## Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)

## XGBoost

In [ ]:
eval_set = [(X_train, Y_train), (X_val, Y_val)]
model = xgb.XGBRegressor(n_estimators=250, max_depth=7, eta=0.01, subsample=1, colsample_bytree=1)
model.fit(X_train, Y_train, eval_metric='rmse', eval_set=eval_set)

In [ ]:
results = model.evals_result()
plt.plot(results['validation_0']['rmse'], label='train')
plt.plot(results['validation_1']['rmse'], label='test')
plt.legend()
plt.show()

In [ ]:
variables_df = pd.DataFrame(feats, columns=['Variables'])
importance_score_df = pd.DataFrame(model.feature_importances_, columns=['Importance Score'])
feature_imp = pd.concat([variables_df, importance_score_df], axis=1)
feature_imp = feature_imp.sort_values(by='Importance Score', ascending=False)
feature_imp = feature_imp.reset_index()
x = 'Importance Score'
y = 'Variables'
sns.barplot(x=x, y=y, data=feature_imp[:20], orient="h", palette="Blues_r")
plt.title('Feature Importance')
plt.show()

## Support Vector Regressor

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
model.fit(X_train, Y_train)

In [ ]:
preds = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(Y_val, preds))
print("rmse=", rmse, "Y_val mean=", Y_val.mean())

tmp = {'Y_test':Y_val,'preds':preds} 
tmp_df = pd.DataFrame(tmp)
tmp_df.sort_values(by='Y_test', inplace=True)
sns.scatterplot(data=tmp_df, x='Y_test', y='preds')

In [ ]:
# Prediting on Null values

test_null = test[test['days'].isna()]
X_test_null = test_null[feats].values
preds_test_null = model.predict(X_test_null)

tmp_df = pd.DataFrame(preds)
print(tmp_df.describe())
tmp_df = pd.DataFrame(preds_test_null)
print(tmp_df.describe())

## Predicting Biological Age

In [ ]:
## Predicting biological age using decay function

data = orig_data.copy()

data['BMI_>30'] = np.where(data['BMI_>30']=='Y', 1, 0)
data.rename(columns={'BMI_>30':'BMI_more_than_30'}, inplace=True)
data['Sex'] = np.where(data['Sex']=='Male', 1, 0)
data['Tobacco'] = np.where(data['Tobacco']=='Yes', 1, 0)

data['Alcohol_abuse'] = np.where(data['Alcohol_abuse'].isna(),0,1)

col = 'FRS_10-year_risk_(%)'
data[col] = data[col].astype(str)
data[col] = data[col].str.replace('X','')
data[col] = data[col].str.replace('<1%',str(np.random.uniform(0, 0.01)))
data[col] = data[col].str.replace('>30%',str(np.random.uniform(0.3, 0.99)))
data[col] = pd.to_numeric(data[col], errors='coerce')
data[col] = data[col].astype(float)
med = data[col].median()
data[col] = data[col].fillna(med)
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)

for col in ['FRAX_10y_Fx_Prob_(Orange-w/_DXA)','FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)']:
    data[col] = data[col].astype(str)
    data[col] = data[col].str.replace('_','')
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col] = data[col].astype(float)
    med = data[col].median()
    data[col] = data[col].fillna(med)
    upper_limit = data[col].quantile(0.95)
    data[col] = data[col].clip(upper_limit)

data['Met_Sx'] = np.where(data['Met_Sx']=='Y', 1, np.where(data['Met_Sx']=='N', 0, -1))


data['CVD'] = np.where(data['CVD_DX'].isna(),0,1)
data['MI'] = np.where(data['MI_DX'].isna(),0,1)
data['Heart_failure'] = np.where(data['Heart_failure_DX'].isna(),0,1)


max_bio_subtract = 80000
decay_rate = 0.0001
max_bio_age = 100

data['Decay_Days'] = 18 + max_bio_subtract*(1 - np.exp(-decay_rate*data['DEATH_[d_from_CT]']))
data['Bio_Age_AT_CT[DAYS]'] = max_bio_age*365 - data['Decay_Days']


data['days'] = data['Bio_Age_AT_CT[DAYS]']

data['diabetes'] = np.where(data['Type_2_Diabetes_DX'].isna(), 0, 1)

data['L1_HU_BMD'] = data['L1_HU_BMD'].clip(lower=0)
data['TAT/Body_Area'] = data['TAT_Area_(cm2)']/data['Total_Body_Area_EA_(cm2)']
col = 'VAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'SAT_Area_(cm2)'
upper_limit = data[col].quantile(0.99)
data[col] = data[col].clip(upper_limit)
col = 'VAT/SAT_Ratio'
upper_limit = data[col].quantile(0.95)
data[col] = data[col].clip(upper_limit)

data['AoCa_Agatston'] = data['AoCa_Agatston'].clip(upper=data['AoCa_Agatston'].quantile(0.99))
data['AoCa_Agatston'] = data['AoCa_Agatston'].fillna(data['AoCa_Agatston'].median())
data['AoCa_Agatston'] = np.log(data['AoCa_Agatston']+1)

data['Liver_HU_(Median)'] = pd.to_numeric(data['Liver_HU_(Median)'], errors='coerce')
data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].astype(float)
#data['Liver_HU_(Median)'] = data['Liver_HU_(Median)'].fillna(data['Liver_HU_(Median)'].median())
                          
for col in ['BMI','TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'VAT_Area_(cm2)','SAT_Area_(cm2)',
            'Muscle_Area_(cm2)','TAT/Body_Area']:
    data[col] = data[col].fillna(data[col].median())


### Below features filled by iterative imputing

impute_it = IterativeImputer()

col = 'L1_HU_BMD'
rel_cols = ['Age_at_CT', 'Muscle_HU', 'L1_HU_BMD']
X = data[rel_cols]
X = impute_it.fit_transform(X)

data[col] = X[:, 2]

### VAT/SAT ratio -- Muscle area, Sex

col = 'VAT/SAT_Ratio'
rel_cols = ['Muscle_Area_(cm2)', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]


col = 'Muscle_HU'
rel_cols = ['Age_at_CT', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]

col = 'L3_SMI_(cm2/m2)'
rel_cols = ['BMI_more_than_30', 'Sex', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]
                       

col = 'Liver_HU_(Median)'                                     
rel_cols = ['BMI_more_than_30', 'TAT/Body_Area', col]
X = data[rel_cols]
X = impute_it.fit_transform(X)
data[col] = X[:,2]


In [ ]:
cols = ['Record_ID', 'Clinical_F/U_interval_[d_from_CT]', 'BMI', 'BMI_more_than_30', 'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse',
       'FRS_10-year_risk_(%)', 'FRAX_10y_Fx_Prob_(Orange-w/_DXA)',
       'FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx',
       'days', 'CVD', 'Heart_failure', 'MI', 'diabetes', 'L1_HU_BMD',
       'TAT_Area_(cm2)', 'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

#print(data['BMI_>30'])
data = data[cols].copy()
data['death_binary'] = np.where(data['days'].isna(),0,1)

In [ ]:
train_idx = pd.read_csv('Patients(RECORD_ID)TrainData.csv')['Record_ID']
test_idx = pd.read_csv('Patients(RECORD_ID)_TestData.csv')['Record_ID']
train = data[data['Record_ID'].isin(train_idx)]
test = data[data['Record_ID'].isin(test_idx)]

In [ ]:
# Train on Non-Null values
train = train[~train['days'].isna()]
val = test[~test['days'].isna()]

In [ ]:
all_feats = ['BMI', 'BMI_more_than_30',
       'Sex', 'Age_at_CT', 'Tobacco', 'Alcohol_abuse', 'FRS_10-year_risk_(%)',
       'FRAX_10y_Fx_Prob_(Orange-w/_DXA)','FRAX_10y_Hip_Fx_Prob_(Orange-w/_DXA)', 'Met_Sx', 'CVD',
       'Heart_failure', 'MI', 'diabetes', 'L1_HU_BMD', 'TAT_Area_(cm2)',
       'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']


ct_feats = ['L1_HU_BMD', 'TAT_Area_(cm2)',
       'Total_Body_Area_EA_(cm2)', 'TAT/Body_Area', 'VAT_Area_(cm2)',
       'SAT_Area_(cm2)', 'VAT/SAT_Ratio', 'Muscle_HU', 'Muscle_Area_(cm2)',
       'L3_SMI_(cm2/m2)', 'AoCa_Agatston', 'Liver_HU_(Median)']

feats = all_feats
#feats = [x for x in all_feats if x not in ct_feats]
#feats = ct_feats

label = 'days'
X_train = train[feats].values
Y_train = train[label].values
X_val = val[feats].values
chrono_val = val['Age_at_CT'].values
Y_val = val[label].values
X_test = test[feats].values
Y_test = test[label].values
chrono_test = test['Age_at_CT'].values

## XGboost

In [ ]:
eval_set = [(X_train, Y_train), (X_val, Y_val)]
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.01, subsample=1, colsample_bytree=1)
model.fit(X_train, Y_train, eval_metric='rmse', eval_set=eval_set)

In [ ]:
results = model.evals_result()
plt.plot(results['validation_0']['rmse'], label='train')
plt.plot(results['validation_1']['rmse'], label='test')
plt.legend()
plt.xlabel('Estimators')
plt.ylabel('RMSE')
plt.show()

In [ ]:
preds = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(Y_val, preds))
print(rmse, Y_val.mean())

print(Y_val.shape)

tmp = {'Y_test':Y_val,'preds':preds, 'chrono': chrono_val} 
tmp_df = pd.DataFrame(tmp)
tmp_df.sort_values(by='Y_test', inplace=True)
sns.scatterplot(data=tmp_df, x='Y_test', y='preds')
plt.show()
sns.distplot(tmp_df['preds']/365)
plt.show()
sns.scatterplot(data=tmp_df, x='chrono', y='preds')
plt.show()